In [14]:
import json

def load_jsonl(file):
    with open(file) as f:
        data = [json.loads(line) for line in f]
    return data

def load_json(file):
    with open(file) as f:
        data = json.loads(f)
    return data

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

cache_dir = "/home/congnguyen/drive/.cache"
hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/albert-xxlarge-v2-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/electra-large-discriminator-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/xlnet-large-cased-snli_mnli_fever_anli_R1_R2_R3-nli"

tokenizer = AutoTokenizer.from_pretrained(hg_model_hub_name, cache_dir=cache_dir)
model = AutoModelForSequenceClassification.from_pretrained(hg_model_hub_name, cache_dir=cache_dir)


tokenizer_config.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49.0/49.0 [00:00<00:00, 153kB/s]
config.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 703/703 [00:00<00:00, 2.23MB/s]
vocab.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 899k/899k [00:00<00:00, 2.64MB/s]
merges.txt: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 456k/456k [00:00<00:00, 907kB/s]
special_tokens_m

In [4]:
if __name__ == '__main__':
    max_length = 256

    premise = "Two women are embracing while holding to go packages."
    hypothesis = "The men are fighting outside a deli."

   
    tokenized_input_seq_pair = tokenizer.encode_plus(premise, hypothesis,
                                                     max_length=max_length,
                                                     return_token_type_ids=True, truncation=True)
    input_ids = torch.Tensor(tokenized_input_seq_pair['input_ids']).long().unsqueeze(0)
    # remember bart doesn't have 'token_type_ids', remove the line below if you are using bart.
    token_type_ids = torch.Tensor(tokenized_input_seq_pair['token_type_ids']).long().unsqueeze(0)
    attention_mask = torch.Tensor(tokenized_input_seq_pair['attention_mask']).long().unsqueeze(0)

    outputs = model(input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    labels=None)
    # Note:
    # "id2label": {
    #     "0": "entailment",
    #     "1": "neutral",
    #     "2": "contradiction"
    # },

    predicted_probability = torch.softmax(outputs[0], dim=1)[0].tolist()  # batch_size only one

    print("Premise:", premise)
    print("Hypothesis:", hypothesis)
    print("Entailment:", predicted_probability[0])
    print("Neutral:", predicted_probability[1])
    print("Contradiction:", predicted_probability[2])


Premise: Two women are embracing while holding to go packages.
Hypothesis: The men are fighting outside a deli.
Entailment: 6.90110246068798e-05
Neutral: 0.0003960769099649042
Contradiction: 0.9995349645614624


In [5]:
data = load_jsonl("../data/COLIEE2024statute_data-English/test.json/riteval_R05_en.jsonl")
data

[{'result': 'Article 537(1) If one of the parties promises in a contract to render a certain performance to a third party, the third party has the right to claim that performance directly from the obligor.\n(2) The validity of the contract referred to in the preceding paragraph is not impaired even if a third party does not exist or a third party is not specified at the time of its formation.\n(3) In the case referred to in paragraph (1), rights of the third party accrue when the third party has manifested intention of availing of the benefit of the contract under that paragraph to the obligor.',
  'content': 'The validity of a third party beneficiary contract made by A with C, the mother of B, to give land X owned by A to B gratuitously, is not impaired even if B is an unborn child as of the time of the formation of the contract.',
  'index': 'R05-01-A',
  'label': 'Y'},
 {'result': 'Article 712 If a minor has inflicted damage on another person but did not have sufficient intellectual

In [12]:
from tqdm import tqdm
count = 0

for item in tqdm(data):
    premise = item["result"]
    hypothesis = item["content"]
    tokenized_input_seq_pair = tokenizer.encode_plus(premise, hypothesis,
                                                     max_length=max_length,
                                                     return_token_type_ids=True, truncation=True)
    input_ids = torch.Tensor(tokenized_input_seq_pair['input_ids']).long().unsqueeze(0)
    # remember bart doesn't have 'token_type_ids', remove the line below if you are using bart.
    token_type_ids = torch.Tensor(tokenized_input_seq_pair['token_type_ids']).long().unsqueeze(0)
    attention_mask = torch.Tensor(tokenized_input_seq_pair['attention_mask']).long().unsqueeze(0)

    outputs = model(input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    labels=None)
    predicted_probability = torch.softmax(outputs[0], dim=1)[0].tolist()
    if predicted_probability[0] > predicted_probability[1] and predicted_probability[0] > predicted_probability[2]:
        label = "Y"
    else:
        label = "N"
    if label == item["label"]:
        count += 1
    if count < 2:
        print(premise)
        print(hypothesis)
        print(predicted_probability)

  1%|█▉                                                                                                                                                                                                              | 1/109 [00:00<00:37,  2.85it/s]

Article 537(1) If one of the parties promises in a contract to render a certain performance to a third party, the third party has the right to claim that performance directly from the obligor.
(2) The validity of the contract referred to in the preceding paragraph is not impaired even if a third party does not exist or a third party is not specified at the time of its formation.
(3) In the case referred to in paragraph (1), rights of the third party accrue when the third party has manifested intention of availing of the benefit of the contract under that paragraph to the obligor.
The validity of a third party beneficiary contract made by A with C, the mother of B, to give land X owned by A to B gratuitously, is not impaired even if B is an unborn child as of the time of the formation of the contract.
[0.9630258679389954, 0.03638390451669693, 0.0005903260898776352]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 109/109 [00:36<00:00,  2.98it/s]


In [13]:
count

62

In [ ]:
data = load_json("../data/COLIEE2024statute_data-English/text/")